In [1]:
# 操作 browser 的 API
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時的例外工具
from selenium.common.exceptions import TimeoutException

# 等待某個元素的出現
from selenium.webdriver.support.ui import WebDriverWait

# 期待元素出現並執行下一個指令
from selenium.webdriver.support import expected_conditions as EC

# 透過什麼方式選取元素
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

# 強制等待
from time import sleep

# 反爬蟲
import undetected_chromedriver as uc

# 執行 command 的時候用的
import os

# 整理 json 使用的工具
import json

# 計算程式執行的時間
import time

# 寫入csv檔
import csv

# 平行任務處理
from concurrent.futures import ThreadPoolExecutor as tpe

from fake_useragent import UserAgent 
ua = UserAgent(cache= True)

my_options = uc.ChromeOptions()

my_options.add_argument("--start-maximized") # 視窗最大化
my_options.add_argument("--incognito") # 無痕
my_options.add_argument("--disable-popup-blocking") # 禁用彈出攔截
my_options.add_argument("--disable-notifiactions") # 關閉推波通知
my_options.add_argument("--lang=zh-TW") # 設定為繁體中文
my_options.add_argument("--headless")
# my_options.add_argument("--user-agent="+ua.random)

# 建立儲存檔案的資料夾
folderPath = "BeitouRestaurant"
if os.path.exists(folderPath) != True:    
    os.makedirs(folderPath)
    
# 每搜尋一次計數器+1
index = 0

In [2]:
def page_link(link):
   
    global index
    
    # 自動取得Chrom driver 不用再額外取得 executable_path="chromedriver.exe"
    driver = uc.Chrome(options=my_options)
    
    # GoogleMap網址
    url = "https://www.google.com.tw/maps/@23.546162,120.6402133,8z?hl=zh-TW"
    
    driver.get(url)
    
    sleep(1)
    
    # 計算每跑一次跑多久
    start = time.time()
        
    # 輸入關鍵字搜尋
    driver.find_element(
        By.CSS_SELECTOR,
        "input#searchboxinput"
    ).send_keys(link)
    
    # 按下搜尋按鈕
    driver.find_element(
        By.CSS_SELECTOR,
        "button#searchbox-searchbutton"
    ).click()
        
    sleep (3)
        
    # 找搜尋元素結果的div
    focus = driver.find_elements(
        By.CSS_SELECTOR,
        "div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd"
    )[1]

    offset = 0
    innerHeight = 0
    count = 0  # 累計無效滾動次數
    limit = 5  # 最大無效滾動次數
    Flag = True
    counter = 0
        
    # 開始滾輪找到最底部
    while Flag:
            
        try:
            # 等待底部元素出現
            WebDriverWait(driver, 0.1).until(
                EC.presence_of_element_located( 
                    (By.CSS_SELECTOR, "span.HlvSq") 
                )
            )
            
            break
            
        except TimeoutException:

           # offset: 拉槓到頁面頂端的距離
            offset = driver.execute_script('return arguments[0].scrollTop', focus)

            # 執行js指令捲動頁面
            driver.execute_script('arguments[0].scrollTo(0, arguments[0].scrollHeight)', focus)
            
            # innerHeight: 頁面高度 = 拉槓到頁面頂端的距離
            innerHeight = driver.execute_script('return arguments[0].scrollHeight = arguments[0].scrollTop', focus)
            
            # 如果「拉槓到頁面頂端的距離」(offset)等於「頁面高度 = 拉槓到頁面頂端的距離」(innerHeight)，代表搜尋可能逾時
            if offset == innerHeight:
                count += 1                
              
            # 計數器等於限制數則搜尋確實逾時了,需重新整理並搜尋
            if count == limit:
                
                counter+=1
                
                print(f"{link}需重新整理,搜尋逾時,本次刷新第{counter}次")
                
                # 網頁重新整理
                driver.execute_script('window.location.replace(window.location.href)')
                
                # 重新搜尋
                driver.find_element(
                    By.CSS_SELECTOR,
                    "button#searchbox-searchbutton"
                 ).click()
                
                sleep(3)

                # 找搜尋結果的div
                focus = driver.find_elements(
                    By.CSS_SELECTOR,
                    "div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd"
                    )[1]

                # 重新定義需要的變數
                offset = 0
                innerHeight = 0
                count = 0  # 累計無效滾動次數
                
            sleep(1)
            
    # 尋找所有元素的href    
    href_ = driver.find_elements(
        By.CSS_SELECTOR,
        "div[data-js-log-root] a[aria-label]"
    )
      
    counter = 0 
    
    # 把所有href寫入csv裡
    for href in href_: 
        with open(f'{folderPath}/{folderPath}writer.csv', 'a', newline='') as f:
            writer = csv.writer(f)
            url = href.get_attribute("href")
            writer.writerow([url])   
        counter+=1
    
    # 清除搜尋的關鍵字
    driver.find_element(
        By.CSS_SELECTOR,
        "input#searchboxinput"
    ).clear()   
    
    # 每搜尋成功並寫入就計數器+1
    index+=1
    
    end = time.time()
    print(f'第{index}筆 {link},總共有:{counter}筆,已寫入完成')
    print("process_time 測量時間：%f 秒" % (end - start))
        
    driver.quit()
    
    # 每條(街or路)的全部restaurantType都抓完,則提示已完成全(街or路)抓取,,總共restaurantType有26種
    if index%26 == 0:
        regex01 = r'.+(街|路)'
        string01 = link
        match01 = re.search(regex01, string01)
        print(f'注意!!!,{match01[0]}:已全數寫入完成\n')


In [3]:
def multiprocess():
    
    district = "北投區" 
    
    roadNames = ["大度路３段","大業路","大興街","天母北路","天母西路","奇岩路","奉賢路","學園路","宜山路","富貴一路","實踐街","尊賢街","崇仁路１段","崇仰一路","崇仰七路","崇仰三路","崇仰九路","崇仰二路","崇仰五路","崇仰六路","崗山路","幽雅路","幽雅路杏林巷","建國街","建民路","復興一路","復興三路","復興二路","復興四路","懷德街","承德路６段","承德路７段","振興街","振華街","文化三路","文林北路","新市街","新民路","新民路康樂巷","新民路香丘巷","新生街","新興路","明德路","杏林一路","杏林三路","杏林二路","東昇路","東華街１段","東華街２段","東陽街","榮華一路","榮華三路","榮華二路","樹林路","民族街","民權街","永興路１段","永興路２段","泉源路","泉源路華南巷","洲美街","清江路","湖山路１段","湖山路２段","湖底路","溫泉路","溫泉路天主巷","溫泉路湯元巷","溫泉路銀光巷","珠海路","登山路","知行路","石仙路","石牌路１段","石牌路２段","磺港路","福美路","秀山路","稻香路","立功街","立德路","立賢路","立農街１段","立農街２段","竹子湖路","紗帽路","義方街","義理街","翠宜路","翠嶺路","翠華街","翠雲街","聖景路","育仁路","自強街","致遠一路１段","致遠一路２段","致遠三路","致遠二路","行義路","裕民一路","裕民三路","裕民二路","裕民六路","裕民四路","西園街","西安街１段","西安街２段","豐年路１段","豐年路２段","進賢路","重三路","長壽路","開明街","關渡路","陽明路１段","陽明路２段","雙全街"]
    
    # 26種
    restaurantType = ["火鍋","拉麵","日式","美式","義式","法式","中式","台式","韓式","德式","地中海料理","印度料理","越式","港式","泰式","南洋","素食","鐵板燒","餐酒館","咖啡廳","熱炒店","早午餐","甜點店","燒肉","海鮮餐廳","牛排"]
    
    drt = []
    
    for road in roadNames:
        for type_ in restaurantType:
            keyword = f'{district}{road}{type_}' 
            # 平行運算x2,
            drt.append(keyword)
    
    with tpe(max_workers=6) as executor:
        executor.map(page_link,drt) # 執行page_link涵式,keyword是涵式需要的變數
        
        
#             with tpe(max_workers=3) as executor:
#                 executor.sumbit(page_link,keyword)
        

In [4]:
if __name__=="__main__":
    multiprocess()

第1筆 北投區大度路３段美式,總共有:12筆,已寫入完成
process_time 測量時間：3.572342 秒
第2筆 北投區大度路３段日式,總共有:25筆,已寫入完成
process_time 測量時間：10.169608 秒
第3筆 北投區大度路３段火鍋,總共有:56筆,已寫入完成
process_time 測量時間：10.628307 秒
第4筆 北投區大度路３段法式,總共有:120筆,已寫入完成
process_time 測量時間：40.955467 秒
第5筆 北投區大度路３段義式,總共有:120筆,已寫入完成
process_time 測量時間：45.230535 秒
第6筆 北投區大度路３段地中海料理,總共有:82筆,已寫入完成
process_time 測量時間：29.470427 秒
第7筆 北投區大度路３段中式,總共有:111筆,已寫入完成
process_time 測量時間：38.976158 秒
第8筆 北投區大度路３段韓式,總共有:120筆,已寫入完成
process_time 測量時間：40.553077 秒
第9筆 北投區大度路３段印度料理,總共有:120筆,已寫入完成
process_time 測量時間：40.858084 秒
第10筆 北投區大度路３段素食,總共有:107筆,已寫入完成
process_time 測量時間：20.022810 秒
第11筆 北投區大度路３段鐵板燒,總共有:59筆,已寫入完成
process_time 測量時間：26.212028 秒
北投區大度路３段餐酒館需重新整理,搜尋逾時,本次刷新第1次
第12筆 北投區大度路３段港式,總共有:120筆,已寫入完成
process_time 測量時間：40.519107 秒
第13筆 北投區大度路３段南洋,總共有:120筆,已寫入完成
process_time 測量時間：40.756498 秒
第14筆 北投區大度路３段泰式,總共有:120筆,已寫入完成
process_time 測量時間：43.122548 秒
第15筆 北投區大度路３段早午餐,總共有:18筆,已寫入完成
process_time 測量時間：4.901753 秒
第16筆 北投區大度路３段咖啡廳,總共有:115筆,已寫入完成
process_time 測量時間：39.177349 秒
第17

北投區奇岩路地中海料理需重新整理,搜尋逾時,本次刷新第1次
北投區天母西路日式需重新整理,搜尋逾時,本次刷新第9次
第120筆 北投區奇岩路越式,總共有:82筆,已寫入完成
process_time 測量時間：30.601806 秒
第121筆 北投區奇岩路印度料理,總共有:120筆,已寫入完成
process_time 測量時間：40.082238 秒
第122筆 北投區奇岩路南洋,總共有:15筆,已寫入完成
process_time 測量時間：4.824163 秒
第123筆 北投區奇岩路港式,總共有:117筆,已寫入完成
process_time 測量時間：39.957120 秒
第124筆 北投區奇岩路地中海料理,總共有:91筆,已寫入完成
process_time 測量時間：63.585377 秒
北投區天母西路日式需重新整理,搜尋逾時,本次刷新第10次
第125筆 北投區奇岩路泰式,總共有:120筆,已寫入完成
process_time 測量時間：50.330966 秒
第126筆 北投區奇岩路鐵板燒,總共有:45筆,已寫入完成
process_time 測量時間：18.740237 秒
北投區奇岩路咖啡廳需重新整理,搜尋逾時,本次刷新第1次
第127筆 北投區奇岩路素食,總共有:120筆,已寫入完成
process_time 測量時間：40.269785 秒
第128筆 北投區奇岩路餐酒館,總共有:124筆,已寫入完成
process_time 測量時間：42.118046 秒
北投區天母西路日式需重新整理,搜尋逾時,本次刷新第11次
第129筆 北投區奇岩路熱炒店,總共有:101筆,已寫入完成
process_time 測量時間：35.605039 秒
第130筆 北投區奇岩路早午餐,總共有:120筆,已寫入完成
process_time 測量時間：40.063845 秒
第131筆 北投區奇岩路甜點店,總共有:102筆,已寫入完成
process_time 測量時間：35.370706 秒
北投區天母西路日式需重新整理,搜尋逾時,本次刷新第12次
北投區奇岩路海鮮餐廳需重新整理,搜尋逾時,本次刷新第1次
第132筆 北投區奇岩路燒肉,總共有:120筆,已寫入完成
process_time 測量時間：40.250396 秒
第133筆 北投區奇岩路牛

第238筆 北投區實踐街越式,總共有:120筆,已寫入完成
process_time 測量時間：40.242094 秒
第239筆 北投區實踐街港式,總共有:120筆,已寫入完成
process_time 測量時間：39.915983 秒
北投區實踐街泰式需重新整理,搜尋逾時,本次刷新第1次
第240筆 北投區實踐街美式,總共有:120筆,已寫入完成
process_time 測量時間：105.355387 秒
第241筆 北投區實踐街南洋,總共有:106筆,已寫入完成
process_time 測量時間：36.867733 秒
第242筆 北投區實踐街素食,總共有:120筆,已寫入完成
process_time 測量時間：40.078934 秒
北投區實踐街餐酒館需重新整理,搜尋逾時,本次刷新第1次
第243筆 北投區實踐街鐵板燒,總共有:120筆,已寫入完成
process_time 測量時間：40.143589 秒
第244筆 北投區實踐街咖啡廳,總共有:120筆,已寫入完成
process_time 測量時間：40.047148 秒
第245筆 北投區實踐街熱炒店,總共有:87筆,已寫入完成
process_time 測量時間：32.785833 秒
第246筆 北投區實踐街早午餐,總共有:120筆,已寫入完成
process_time 測量時間：40.075443 秒
北投區實踐街甜點店需重新整理,搜尋逾時,本次刷新第1次
北投區實踐街餐酒館需重新整理,搜尋逾時,本次刷新第2次
北投區實踐街燒肉需重新整理,搜尋逾時,本次刷新第1次
第247筆 北投區尊賢街拉麵,總共有:21筆,已寫入完成
process_time 測量時間：4.962215 秒
北投區實踐街海鮮餐廳需重新整理,搜尋逾時,本次刷新第1次
第248筆 北投區實踐街牛排,總共有:120筆,已寫入完成
process_time 測量時間：40.043138 秒
第249筆 北投區尊賢街火鍋,總共有:118筆,已寫入完成
process_time 測量時間：40.054885 秒
北投區實踐街餐酒館需重新整理,搜尋逾時,本次刷新第3次
第250筆 北投區實踐街燒肉,總共有:120筆,已寫入完成
process_time 測量時間：71.319484 秒
第251筆 北投區尊賢街日式,總共有:120筆

北投區崇仰七路餐酒館需重新整理,搜尋逾時,本次刷新第5次
第353筆 北投區崇仰三路越式,總共有:82筆,已寫入完成
process_time 測量時間：28.860430 秒
第354筆 北投區崇仰三路印度料理,總共有:120筆,已寫入完成
process_time 測量時間：40.019622 秒
北投區崇仰一路法式需重新整理,搜尋逾時,本次刷新第16次
北投區崇仰三路泰式需重新整理,搜尋逾時,本次刷新第1次
第355筆 北投區崇仰三路港式,總共有:120筆,已寫入完成
process_time 測量時間：40.356689 秒
第356筆 北投區崇仰三路南洋,總共有:52筆,已寫入完成
process_time 測量時間：19.142982 秒
北投區崇仰七路餐酒館需重新整理,搜尋逾時,本次刷新第6次
第357筆 北投區崇仰三路鐵板燒,總共有:28筆,已寫入完成
process_time 測量時間：11.244549 秒
北投區崇仰一路法式需重新整理,搜尋逾時,本次刷新第17次
第358筆 北投區崇仰三路素食,總共有:120筆,已寫入完成
process_time 測量時間：40.059202 秒
第359筆 北投區崇仰三路泰式,總共有:120筆,已寫入完成
process_time 測量時間：69.758321 秒
第360筆 北投區崇仰三路咖啡廳,總共有:52筆,已寫入完成
process_time 測量時間：19.143077 秒
第361筆 北投區崇仰三路熱炒店,總共有:34筆,已寫入完成
process_time 測量時間：12.824412 秒
北投區崇仰三路餐酒館需重新整理,搜尋逾時,本次刷新第1次
北投區崇仰七路餐酒館需重新整理,搜尋逾時,本次刷新第7次
北投區崇仰一路法式需重新整理,搜尋逾時,本次刷新第18次
第362筆 北投區崇仰三路甜點店,總共有:38筆,已寫入完成
process_time 測量時間：14.364769 秒
北投區崇仰一路法式需重新整理,搜尋逾時,本次刷新第19次
第363筆 北投區崇仰三路早午餐,總共有:120筆,已寫入完成
process_time 測量時間：40.056047 秒
北投區崇仰三路餐酒館需重新整理,搜尋逾時,本次刷新第2次
第364筆 北投區崇仰三路燒肉,總共有:120筆,已寫入完成
process_

北投區崗山路義式需重新整理,搜尋逾時,本次刷新第1次
第462筆 北投區崗山路德式,總共有:19筆,已寫入完成
process_time 測量時間：4.909192 秒
第463筆 北投區崗山路法式,總共有:120筆,已寫入完成
process_time 測量時間：41.848999 秒
第464筆 北投區崗山路中式,總共有:120筆,已寫入完成
process_time 測量時間：39.967927 秒
第465筆 北投區崗山路美式,總共有:124筆,已寫入完成
process_time 測量時間：71.527856 秒
北投區崗山路地中海料理需重新整理,搜尋逾時,本次刷新第1次
第466筆 北投區崗山路韓式,總共有:120筆,已寫入完成
process_time 測量時間：40.215053 秒
第467筆 北投區崗山路義式,總共有:120筆,已寫入完成
process_time 測量時間：73.263098 秒
第468筆 北投區崗山路南洋,總共有:7筆,已寫入完成
process_time 測量時間：4.765032 秒
北投區崗山路港式需重新整理,搜尋逾時,本次刷新第1次
第469筆 北投區崗山路印度料理,總共有:120筆,已寫入完成
process_time 測量時間：40.345567 秒
第470筆 北投區崗山路越式,總共有:120筆,已寫入完成
process_time 測量時間：40.173326 秒
北投區崗山路泰式需重新整理,搜尋逾時,本次刷新第1次
第471筆 北投區崗山路地中海料理,總共有:120筆,已寫入完成
process_time 測量時間：71.869607 秒
第472筆 北投區崗山路素食,總共有:120筆,已寫入完成
process_time 測量時間：40.220697 秒
第473筆 北投區崗山路鐵板燒,總共有:82筆,已寫入完成
process_time 測量時間：30.329022 秒
北投區崗山路餐酒館需重新整理,搜尋逾時,本次刷新第1次
第474筆 北投區崗山路港式,總共有:122筆,已寫入完成
process_time 測量時間：77.770273 秒
第475筆 北投區崗山路咖啡廳,總共有:120筆,已寫入完成
process_time 測量時間：40.128324 秒
第476筆 北投區崗山路熱炒店,總共有:

...

第2153筆 北投區自強街餐酒館,總共有:120筆,已寫入完成
process_time 測量時間：69.793304 秒
第2154筆 北投區致遠一路１段拉麵,總共有:60筆,已寫入完成
process_time 測量時間：20.810574 秒
第2155筆 北投區致遠一路１段火鍋,總共有:95筆,已寫入完成
process_time 測量時間：32.240412 秒
北投區致遠一路１段美式需重新整理,搜尋逾時,本次刷新第1次
第2156筆 北投區自強街牛排,總共有:120筆,已寫入完成
process_time 測量時間：40.090560 秒
第2157筆 北投區致遠一路１段台式,總共有:2筆,已寫入完成
process_time 測量時間：3.189038 秒
第2158筆 北投區致遠一路１段中式,總共有:24筆,已寫入完成
process_time 測量時間：11.702825 秒
第2159筆 北投區致遠一路１段義式,總共有:70筆,已寫入完成
process_time 測量時間：25.614216 秒
第2160筆 北投區致遠一路１段地中海料理,總共有:1筆,已寫入完成
process_time 測量時間：3.195291 秒
第2161筆 北投區致遠一路１段德式,總共有:15筆,已寫入完成
process_time 測量時間：6.448448 秒
第2162筆 北投區致遠一路１段美式,總共有:52筆,已寫入完成
process_time 測量時間：49.035841 秒
第2163筆 北投區致遠一路１段法式,總共有:86筆,已寫入完成
process_time 測量時間：32.098937 秒
第2164筆 北投區致遠一路１段南洋,總共有:28筆,已寫入完成
process_time 測量時間：6.555109 秒
第2165筆 北投區致遠一路１段印度料理,總共有:64筆,已寫入完成
process_time 測量時間：22.370106 秒
第2166筆 北投區致遠一路１段港式,總共有:69筆,已寫入完成
process_time 測量時間：25.567949 秒
第2167筆 北投區致遠一路１段韓式,總共有:120筆,已寫入完成
process_time 測量時間：40.023260 秒
第2168筆 北投區致遠一路１段越式,總共有:183筆,

第2274筆 北投區裕民一路拉麵,總共有:120筆,已寫入完成
process_time 測量時間：39.920104 秒
第2275筆 北投區裕民一路美式,總共有:102筆,已寫入完成
process_time 測量時間：35.493135 秒
第2276筆 北投區裕民一路義式,總共有:120筆,已寫入完成
process_time 測量時間：40.226640 秒
第2277筆 北投區裕民一路法式,總共有:120筆,已寫入完成
process_time 測量時間：40.051826 秒
第2278筆 北投區裕民一路地中海料理,總共有:21筆,已寫入完成
process_time 測量時間：9.617798 秒
第2279筆 北投區裕民一路德式,總共有:38筆,已寫入完成
process_time 測量時間：14.344389 秒
北投區裕民一路韓式需重新整理,搜尋逾時,本次刷新第1次
第2280筆 北投區裕民一路中式,總共有:120筆,已寫入完成
process_time 測量時間：40.040838 秒
第2281筆 北投區裕民一路泰式,總共有:42筆,已寫入完成
process_time 測量時間：12.406451 秒
第2282筆 北投區裕民一路印度料理,總共有:120筆,已寫入完成
process_time 測量時間：40.143431 秒
第2283筆 北投區裕民一路越式,總共有:120筆,已寫入完成
process_time 測量時間：40.122257 秒
第2284筆 北投區裕民一路港式,總共有:120筆,已寫入完成
process_time 測量時間：40.111695 秒
第2285筆 北投區裕民一路韓式,總共有:120筆,已寫入完成
process_time 測量時間：70.922584 秒
第2286筆 北投區裕民一路鐵板燒,總共有:45筆,已寫入完成
process_time 測量時間：16.067760 秒
第2287筆 北投區裕民一路南洋,總共有:117筆,已寫入完成
process_time 測量時間：40.036610 秒
第2288筆 北投區裕民一路素食,總共有:116筆,已寫入完成
process_time 測量時間：40.162908 秒
北投區裕民一路咖啡廳需重新整理,搜尋逾時,本次刷新第1次
第2289筆 北投區裕民

第2401筆 北投區西園街地中海料理,總共有:55筆,已寫入完成
process_time 測量時間：23.290447 秒
第2402筆 北投區西園街印度料理,總共有:120筆,已寫入完成
process_time 測量時間：40.257617 秒
第2403筆 北投區西園街越式,總共有:120筆,已寫入完成
process_time 測量時間：40.087736 秒
第2404筆 北投區西園街港式,總共有:120筆,已寫入完成
process_time 測量時間：40.071232 秒
第2405筆 北投區西園街泰式,總共有:120筆,已寫入完成
process_time 測量時間：40.119385 秒
第2406筆 北投區西園街南洋,總共有:120筆,已寫入完成
process_time 測量時間：40.191835 秒
第2407筆 北投區西園街素食,總共有:120筆,已寫入完成
process_time 測量時間：39.862191 秒
北投區西園街餐酒館需重新整理,搜尋逾時,本次刷新第1次
第2408筆 北投區西園街鐵板燒,總共有:120筆,已寫入完成
process_time 測量時間：40.130058 秒
第2409筆 北投區西園街咖啡廳,總共有:120筆,已寫入完成
process_time 測量時間：40.386058 秒
北投區西園街甜點店需重新整理,搜尋逾時,本次刷新第1次
第2410筆 北投區西園街熱炒店,總共有:120筆,已寫入完成
process_time 測量時間：40.119227 秒
北投區西園街燒肉需重新整理,搜尋逾時,本次刷新第1次
第2411筆 北投區西安街１段拉麵,總共有:29筆,已寫入完成
process_time 測量時間：11.176394 秒
第2412筆 北投區西園街早午餐,總共有:120筆,已寫入完成
process_time 測量時間：50.216714 秒
第2413筆 北投區西園街海鮮餐廳,總共有:103筆,已寫入完成
process_time 測量時間：35.642770 秒
第2414筆 北投區西安街１段火鍋,總共有:95筆,已寫入完成
process_time 測量時間：31.871954 秒
第2415筆 北投區西安街１段日式,總共有:48筆,已寫入完成
process_time 測量時間：1

第2527筆 北投區進賢路素食,總共有:120筆,已寫入完成
process_time 測量時間：40.389093 秒
第2528筆 北投區進賢路咖啡廳,總共有:38筆,已寫入完成
process_time 測量時間：14.440708 秒
北投區進賢路餐酒館需重新整理,搜尋逾時,本次刷新第1次
第2529筆 北投區進賢路港式,總共有:120筆,已寫入完成
process_time 測量時間：77.571863 秒
北投區進賢路熱炒店需重新整理,搜尋逾時,本次刷新第1次
第2530筆 北投區進賢路甜點店,總共有:92筆,已寫入完成
process_time 測量時間：32.229270 秒
第2531筆 北投區進賢路早午餐,總共有:120筆,已寫入完成
process_time 測量時間：40.058576 秒
第2532筆 北投區進賢路燒肉,總共有:120筆,已寫入完成
process_time 測量時間：39.999806 秒
第2533筆 北投區進賢路餐酒館,總共有:120筆,已寫入完成
process_time 測量時間：68.593150 秒
第2534筆 北投區進賢路海鮮餐廳,總共有:102筆,已寫入完成
process_time 測量時間：43.546104 秒
北投區重三路火鍋需重新整理,搜尋逾時,本次刷新第1次
第2535筆 北投區進賢路牛排,總共有:120筆,已寫入完成
process_time 測量時間：40.131737 秒
第2536筆 北投區重三路拉麵,總共有:105筆,已寫入完成
process_time 測量時間：35.345874 秒
第2537筆 北投區重三路日式,總共有:120筆,已寫入完成
process_time 測量時間：40.051332 秒
第2538筆 北投區重三路美式,總共有:120筆,已寫入完成
process_time 測量時間：39.929532 秒
第2539筆 北投區重三路義式,總共有:98筆,已寫入完成
process_time 測量時間：33.681024 秒
第2540筆 北投區重三路德式,總共有:20筆,已寫入完成
process_time 測量時間：4.878427 秒
第2541筆 北投區重三路火鍋,總共有:120筆,已寫入完成
process_time 測量時間：71.592505 秒
第

第2646筆 北投區陽明路２段地中海料理,總共有:68筆,已寫入完成
process_time 測量時間：27.586231 秒
第2647筆 北投區陽明路２段印度料理,總共有:104筆,已寫入完成
process_time 測量時間：35.379167 秒
第2648筆 北投區陽明路２段越式,總共有:120筆,已寫入完成
process_time 測量時間：40.054422 秒
第2649筆 北投區陽明路２段港式,總共有:120筆,已寫入完成
process_time 測量時間：40.317060 秒
第2650筆 北投區陽明路２段泰式,總共有:120筆,已寫入完成
process_time 測量時間：40.088615 秒
北投區陽明路２段鐵板燒需重新整理,搜尋逾時,本次刷新第1次
第2651筆 北投區陽明路２段南洋,總共有:120筆,已寫入完成
process_time 測量時間：40.112399 秒
第2652筆 北投區陽明路２段素食,總共有:120筆,已寫入完成
process_time 測量時間：40.190496 秒
北投區陽明路２段咖啡廳需重新整理,搜尋逾時,本次刷新第1次
第2653筆 北投區陽明路２段餐酒館,總共有:120筆,已寫入完成
process_time 測量時間：40.044923 秒
第2654筆 北投區陽明路２段熱炒店,總共有:120筆,已寫入完成
process_time 測量時間：40.071796 秒
北投區陽明路２段甜點店需重新整理,搜尋逾時,本次刷新第1次
北投區陽明路２段鐵板燒需重新整理,搜尋逾時,本次刷新第2次
第2655筆 北投區陽明路２段早午餐,總共有:120筆,已寫入完成
process_time 測量時間：40.313456 秒
第2656筆 北投區陽明路２段燒肉,總共有:120筆,已寫入完成
process_time 測量時間：40.194975 秒
第2657筆 北投區陽明路２段海鮮餐廳,總共有:120筆,已寫入完成
process_time 測量時間：40.335139 秒
第2658筆 北投區陽明路２段甜點店,總共有:120筆,已寫入完成
process_time 測量時間：69.633841 秒
第2659筆 北投區陽明路２段牛排,總共有:120筆,已寫入完成
process_time 測量時間：